## Great Circle Calculation
- [gc_aangle](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_aangle.shtml):  Finds the acute angle between two great circles on the globe. 
- [gc_clkwise](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_clkwise.shtml):  Tests clockwise/counterclockwise ordering of points on spherical polygon. 
- [gc_dangle](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_dangle.shtml):  Finds the directed angle between two great circles having a specified intersection point. 
- [gc_inout](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_inout.shtml):  Determines if a list of lat/lon specified points are inside or outside of spherical lat/lon polygon(s). 
- [gc_latlon](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_latlon.shtml): Finds the great circle distance (true surface distance) between two points on the globe and interpolates points along the great circle. 
- [gc_onarc](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_onarc.shtml):  Determines if a point on the globe lies on a specified great circle arc. 
- [gc_pnt2gc](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_pnt2gc.shtml):  Finds the angular distance from a point to a great circle. 
- [gc_qarea](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_qarea.shtml):  Finds the area of a quadrilateral patch on the unit sphere. 
- [gc_tarea](https://www.ncl.ucar.edu/Document/Functions/Built-in/gc_tarea.shtml):  Finds the area of a triangular patch on the unit sphere. 

In [29]:
from pyproj import Geod
geodesic = Geod(ellps="WGS84")

## gc_aangle

This function finds the acute angle between two great circles, given two pairs of points on the globe that define two arcs. 

### Input:
Latitudes and longitudes, in degrees, of vertices. The first two pairs of lat/lon values specify vertices of an arc of one great circle and the second two pairs specify the vertices of the second circle. These can be multi-dimensional arrays, but the rightmost dimension size must be 4 for specifying the vertices of the two arcs. If lat and lon have more than one dimension, then they must agree in number of dimensions and dimension sizes.
Return value
### Output
The acute angle, in degrees, between the great circles defined by the arcs as describe above. This will be a non-negative value. If the input arrays are singly dimensioned, then the return value will be a scalar. If the input arrays are multi-dimensional, then the returned array will have one less dimension than the number of dimensions of the input arrays and the dimension sizes of the returned array will agree with those of the input arrays up through their penultimate dimension. The return value will be of type double if either of the input arguments is of type double and type float otherwise.

In [57]:
aangle = ((0.0,  0.0, 0.0,  10.0), (0.0, 10.0, 0.0,   0.0))
arc_1_start  = (aangle[0][0], aangle[1][0])
arc_1_end    = (aangle[0][1], aangle[1][1])
arc_2_start  = (aangle[0][2], aangle[1][2])
arc_2_end    = (aangle[0][3], aangle[1][3])

print(arc_1_start) # (lat, lon)
print(arc_1_end)   # (lat, lon)

print(arc_2_end)   # (lat, lon)
print(arc_2_start) # (lat, lon)

#fwd_bearing, rvs_bearing, distance_m = geodesic.inv(lon_start, lat_start, lon_end, lat_end)
fwd_bearing, rvs_bearing, distance_m = geodesic.inv(arc_1_start[1],
                                                    arc_1_start[0],
                                                    arc_1_end[1],
                                                    arc_1_end[0])
print(f"forward bearing = {fwd_bearing} degrees")
print(f"reverse bearing = {rvs_bearing} degrees")
print(f"distance_m = {distance_m/1000} km")

(0.0, 0.0)
(0.0, 10.0)
(10.0, 0.0)
(0.0, 0.0)
forward bearing = 90.0 degrees
reverse bearing = -90.0 degrees
distance_m = 1113.1949079327358 km
